In [ ]:
import os
import tensorflow as tf

import FFBP.utils as utils
from FFBP.constructors import InputData, BasicLayer, FFBPModel

print(os.getcwd())

In [ ]:
inp_size = (8,4)
targ_size = 36
data_len = 32

num_epochs = 3000
batch_size = 1

eight_things_graph = tf.Graph()

with eight_things_graph.as_default():
    with tf.name_scope('train_data'):
        train_data = InputData(
            path_to_data_file = 'eight_things-train_data.txt',
            num_epochs = num_epochs,
            batch_size = batch_size, 
            inp_size = inp_size, 
            targ_size = targ_size,
            data_len = data_len,
            shuffle = True, 
            shuffle_seed = 1
        )

    with tf.name_scope('test_data'):
        test_data = InputData(
            path_to_data_file = 'eight_things-train_data.txt',
            num_epochs = num_epochs,
            batch_size = data_len,
            inp_size = inp_size, 
            targ_size = targ_size,
            data_len = data_len,
            shuffle = False
        )

    model_name = 'eight_things_model'
    with tf.name_scope(model_name):

        item_inp  = tf.placeholder(dtype = tf.float32, shape=[None, inp_size[0]], name='item_inp')
        relation_inp = tf.placeholder(dtype = tf.float32, shape=[None, inp_size[1]], name='relation_inp')

        representation_layer = BasicLayer(
            layer_name = 'representation_layer', 
            layer_input = item_inp, 
            size = 8, 
            wrange = [-.45, .45], 
            nonlin=tf.nn.sigmoid, 
            bias=True, 
            seed=1, # Use None for random seed value
        )

        hidden_layer = BasicLayer(
            layer_name = 'hidden_layer', 
            layer_input = (representation_layer.output, relation_inp), 
            size = 12, 
            wrange = [-.45, .45], 
            nonlin=tf.nn.sigmoid, 
            bias=True, 
            seed=1, # Use None for random seed value
        )

        attribute_layer = BasicLayer(
            layer_name = 'attribute_layer', 
            layer_input = hidden_layer.output, 
            size = 36, 
            wrange = [-.45, .45], 
            nonlin=tf.nn.sigmoid, 
            bias=True, 
            seed=1, # Use None for random seed value
        )

        target = tf.placeholder(dtype = tf.float32, shape=[None, targ_size], name='targets')

        eight_things_model = FFBPModel(
            name = model_name,
            layers = [representation_layer, hidden_layer, attribute_layer],
            train_data = train_data, 
            inp        = [item_inp, relation_inp],
            targ       = target,
            loss       = tf.reduce_sum(tf.squared_difference(target, attribute_layer.output), name='loss_function'),
            optimizer  = tf.train.MomentumOptimizer(.1, 0),
            test_data = test_data
        )

test_epochs = [i for i in range(0,num_epochs,500)]
print(test_epochs)
        
with tf.Session(graph=eight_things_graph) as sess:
    # initialize variables
    print('initializing variables')
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())

    # create coordinator and queue runners
    print('creating coordinater and starting queue runners')
    coordinator = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coordinator)
    
    print('running model')
    for i in range(num_epochs):
        if any([i==test_epoch for test_epoch in test_epochs]): 
            print('Running Test...')
            eight_things_model.test_epoch(session=sess, verbose=True)
        
        loss = eight_things_model.train_epoch(session=sess, verbose=False)

        if loss < 0.05:
            eight_things_model.test_epoch(session=sess, verbose=True)
            print('Stopped due to loss < 0.05')
            break
    
    coordinator.request_stop()
    coordinator.join(threads)

In [ ]:
# for i,op in enumerate(eight_things_graph.get_operations()):
#     print('{}) {}'.format(i, str(op.name)))
500 * 32